In [83]:
import configparser
from configparser import SafeConfigParser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.types import TimestampType, DateType
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, dayofweek, date_format

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

No module named 'spark'
Traceback (most recent call last):
ModuleNotFoundError: No module named 'spark'



In [2]:
spark = SparkSession \
        .builder \
        .appName("Sparkify_Lake") \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .config("spark.hadoop.fs.s3a.fast.upload","true")\
        .getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
config = SafeConfigParser()
config.read('work/dl.cfg')
print(config.sections())
# a=config.get('KEY')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[]

In [5]:
os.environ['AWS_ACCESS_KEY_ID']=''
os.environ['AWS_SECRET_ACCESS_KEY']=''

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
load_songs = "s3a://udacity-dend/song_data/A/A/A/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [69]:
input_data = "s3a://udacity-dend/"
load_logs = input_data + 'log_data/*/*/*.json'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [78]:
song_data = spark.read.json(load_songs)
song_data.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)

In [70]:
log_data = spark.read.json(load_logs)
log_data.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)

In [17]:
songs_table = song_data[['song_id', 'title', 'artist_id', 'year','duration']]
songs_table.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------------+------------------+----+---------+
|           song_id|               title|         artist_id|year| duration|
+------------------+--------------------+------------------+----+---------+
|SOAFBCP12A8C13CC7D|King Of Scurf (20...|ARTC1LV1187B9A4858|1972|301.40036|
|SOKTJDS12AF72A25E5|Drown In My Own T...|ARA23XO1187B9AF18F|   0|  192.522|
|SOEKAZG12AB018837E|I'll Slap Your Fa...|ARSVTNL1187B992A91|2001|129.85424|
|SOQPWCR12A6D4FB2A3|A Poor Recipe For...|AR73AIO1187B9AD57B|2005|118.07302|
|SOBRKGM12A8C139EF6|Welcome to the Pl...|ARXQBR11187B98A2CC|1985|821.05424|
|SORRNOC12AB017F52B|The Last Beat Of ...|ARSZ7L31187FB4E610|2004|337.81506|
|SOHKNRJ12A6701D1F8|        Drop of Rain|AR10USD1187B99F3F1|   0|189.57016|
|SOAPERH12A58A787DC|The One And Only ...|ARZ5H0P1187B98A1DD|   0|230.42567|
|SOSMJFC12A8C13DE0C|Is That All There...|AR1KTV21187B9ACD72|   0|343.87546|
|SOOVHYF12A8C134892|     I'll Be Waiting|ARCLYBR1187FB53913|1989|304.56118|
|SOERIDA12A6

In [57]:
artists_table = song_data[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']]
artists_table.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------------+--------------------+---------------+----------------+
|         artist_id|         artist_name|     artist_location|artist_latitude|artist_longitude|
+------------------+--------------------+--------------------+---------------+----------------+
|ARTC1LV1187B9A4858|  The Bonzo Dog Band|Goldsmith's Colle...|        51.4536|        -0.01802|
|ARA23XO1187B9AF18F|     The Smithereens|Carteret, New Jersey|       40.57885|       -74.21956|
|ARSVTNL1187B992A91|       Jonathan King|     London, England|       51.50632|        -0.12714|
|AR73AIO1187B9AD57B|   Western Addiction|   San Francisco, CA|       37.77916|      -122.42005|
|ARXQBR11187B98A2CC|Frankie Goes To H...|  Liverpool, England|           null|            null|
|ARSZ7L31187FB4E610|           Devotchka|          Denver, CO|       39.74001|      -104.99226|
|AR10USD1187B99F3F1|Tweeterfriendly M...|Burlington, Ontar...|           null|            null|
|ARZ5H0P1187B98A1DD|          Snoop Dogg

In [64]:
songs_table.write.mode("overwrite").partitionBy("year", "artist_id").parquet("s3a://techlake/output/songs.parquet")
songs_par = spark.read.parquet('s3a://techlake/output/songs.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [81]:
songs_par = spark.read.parquet('s3a://techlake/output/songs.parquet')
songs_par.createOrReplaceTempView("songs_view")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [100]:
s_view = spark.sql("SELECT * FROM songs_view")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'java.lang.RuntimeException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient;'
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/session.py", line 767, in sql
    return DataFrame(self._jsparkSession.sql(sqlQuery), self._wrapped)
  File "/usr/lib/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 69, in deco
    raise AnalysisException(s.split(': ', 1)[1], stackTrace)
pyspark.sql.utils.AnalysisException: 'java.lang.RuntimeException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient;'



In [68]:
artists_table.write.mode("overwrite").parquet("s3a://techlake/output/artists.parquet")
artists_par = spark.read.parquet('s3a://techlake/output/artists.parquet').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------------+--------------------+---------------+----------------+
|         artist_id|         artist_name|     artist_location|artist_latitude|artist_longitude|
+------------------+--------------------+--------------------+---------------+----------------+
|ARTC1LV1187B9A4858|  The Bonzo Dog Band|Goldsmith's Colle...|        51.4536|        -0.01802|
|ARA23XO1187B9AF18F|     The Smithereens|Carteret, New Jersey|       40.57885|       -74.21956|
|ARSVTNL1187B992A91|       Jonathan King|     London, England|       51.50632|        -0.12714|
|AR73AIO1187B9AD57B|   Western Addiction|   San Francisco, CA|       37.77916|      -122.42005|
|ARXQBR11187B98A2CC|Frankie Goes To H...|  Liverpool, England|           null|            null|
|ARSZ7L31187FB4E610|           Devotchka|          Denver, CO|       39.74001|      -104.99226|
|AR10USD1187B99F3F1|Tweeterfriendly M...|Burlington, Ontar...|           null|            null|
|ARZ5H0P1187B98A1DD|          Snoop Dogg

In [98]:
get_timestamp = udf(lambda x: datetime.fromtimestamp(int(int(x) / 1000)), TimestampType())
convert_time = log_data.select('*', get_timestamp(log_data.ts).alias('start_time'))
convert_time.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+------+-------------------+
|              artist|      auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page|     registration|sessionId|                song|status|           ts|           userAgent|userId|         start_time|
+--------------------+----------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+------+-------------------+
|            Harmonia| Logged In|     Ryan|     M|            0|   Smith|655.77751| free|San Jose-Sunnyval...|   PUT|NextSong|1.541016707796E12|      583|       Sehr kosmisch|   200|1542241826796|"Mozilla/5.0 (X11...|    26|2018-11-15 00:30:26|
|         The Prodig

In [99]:
convert_time = convert_time.select('*').filter(convert_time.page == 'NextSong').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+------+-------------------+
|              artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page|     registration|sessionId|                song|status|           ts|           userAgent|userId|         start_time|
+--------------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+------+-------------------+
|            Harmonia|Logged In|     Ryan|     M|            0|   Smith|655.77751| free|San Jose-Sunnyval...|   PUT|NextSong|1.541016707796E12|      583|       Sehr kosmisch|   200|1542241826796|"Mozilla/5.0 (X11...|    26|2018-11-15 00:30:26|
|         The Prodigy|Lo

In [88]:
log_data.createOrReplaceTempView("convert_time_log")
song_data.createOrReplaceTempView("song_data_table")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [89]:
songplays = spark.sql('''
                        SELECT 
                               ct.userid as user_id,
                               level,
                               s.song_id,
                               s.artist_id,
                               ct.sessionId,
                               ct.location,
                               ct.userAgent as user_agent
                          FROM convert_time_log as ct
                          LEFT JOIN song_data_table as s
                            ON ct.artist = s.artist_name
                         WHERE ct.page = "NextSong"
                      ''').write.format('parquet').partitionBy("level").save('s3a://techlake/output/s')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'java.lang.RuntimeException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient;'
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/session.py", line 767, in sql
    return DataFrame(self._jsparkSession.sql(sqlQuery), self._wrapped)
  File "/usr/lib/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 69, in deco
    raise AnalysisException(s.split(': ', 1)[1], stackTrace)
pyspark.sql.utils.AnalysisException: 'java.lang.RuntimeException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient;'



In [97]:
users = log_data[['userId', 'firstName', 'lastName', 'gender', 'level']]
users.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+---------+--------+------+-----+
|userId|firstName|lastName|gender|level|
+------+---------+--------+------+-----+
|    26|     Ryan|   Smith|     M| free|
|    26|     Ryan|   Smith|     M| free|
|    26|     Ryan|   Smith|     M| free|
|     9|    Wyatt|   Scott|     M| free|
|    12|   Austin| Rosales|     M| free|
|    61|   Samuel|Gonzalez|     M| free|
|    61|   Samuel|Gonzalez|     M| free|
|      |     null|    null|  null| paid|
|    80|    Tegan|  Levine|     F| paid|
|    80|    Tegan|  Levine|     F| paid|
|    80|    Tegan|  Levine|     F| paid|
|    80|    Tegan|  Levine|     F| paid|
|    80|    Tegan|  Levine|     F| paid|
|    80|    Tegan|  Levine|     F| paid|
|    80|    Tegan|  Levine|     F| paid|
|    80|    Tegan|  Levine|     F| paid|
|    80|    Tegan|  Levine|     F| paid|
|    15|     Lily|    Koch|     F| paid|
|    80|    Tegan|  Levine|     F| paid|
|    15|     Lily|    Koch|     F| paid|
+------+---------+--------+------+-----+
only showing top

In [96]:
users.write.mode("overwrite").parquet("users.parquet")
users_par = spark.read.parquet('users.parquet')
users_par.select('*').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+---------+--------+------+-----+
|userId|firstName|lastName|gender|level|
+------+---------+--------+------+-----+
|    26|     Ryan|   Smith|     M| free|
|    26|     Ryan|   Smith|     M| free|
|    26|     Ryan|   Smith|     M| free|
|     9|    Wyatt|   Scott|     M| free|
|    12|   Austin| Rosales|     M| free|
|    61|   Samuel|Gonzalez|     M| free|
|    61|   Samuel|Gonzalez|     M| free|
|      |     null|    null|  null| paid|
|    80|    Tegan|  Levine|     F| paid|
|    80|    Tegan|  Levine|     F| paid|
|    80|    Tegan|  Levine|     F| paid|
|    80|    Tegan|  Levine|     F| paid|
|    80|    Tegan|  Levine|     F| paid|
|    80|    Tegan|  Levine|     F| paid|
|    80|    Tegan|  Levine|     F| paid|
|    80|    Tegan|  Levine|     F| paid|
|    80|    Tegan|  Levine|     F| paid|
|    15|     Lily|    Koch|     F| paid|
|    80|    Tegan|  Levine|     F| paid|
|    15|     Lily|    Koch|     F| paid|
+------+---------+--------+------+-----+
only showing top

In [95]:
# year, month, dayofmonth, hour, weekofyear, date_format
time_data = log_data.select('ts', get_timestamp(log_data.ts).alias('start_time'))
                    
time = time_data.select(
                        date_format('start_time', 'HH:mm:ss').alias('start_time'),\
                        hour('start_time').alias('hour'),\
                        dayofmonth('start_time').alias('day'),\
                        weekofyear('start_time').alias('week'),\
                        month('start_time').alias('month'),\
                        year('start_time').alias('year'),\
                        dayofweek('start_time').alias('weekday')
                        )
time.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----+---+----+-----+----+-------+
|start_time|hour|day|week|month|year|weekday|
+----------+----+---+----+-----+----+-------+
|  00:30:26|   0| 15|  46|   11|2018|      5|
|  00:41:21|   0| 15|  46|   11|2018|      5|
|  00:45:41|   0| 15|  46|   11|2018|      5|
|  01:57:51|   1| 15|  46|   11|2018|      5|
|  03:29:37|   3| 15|  46|   11|2018|      5|
|  03:44:09|   3| 15|  46|   11|2018|      5|
|  03:44:20|   3| 15|  46|   11|2018|      5|
|  05:34:34|   5| 15|  46|   11|2018|      5|
|  05:37:57|   5| 15|  46|   11|2018|      5|
|  05:48:55|   5| 15|  46|   11|2018|      5|
|  05:53:44|   5| 15|  46|   11|2018|      5|
|  05:55:56|   5| 15|  46|   11|2018|      5|
|  06:01:02|   6| 15|  46|   11|2018|      5|
|  06:01:53|   6| 15|  46|   11|2018|      5|
|  06:07:37|   6| 15|  46|   11|2018|      5|
|  06:10:33|   6| 15|  46|   11|2018|      5|
|  06:13:54|   6| 15|  46|   11|2018|      5|
|  06:14:16|   6| 15|  46|   11|2018|      5|
|  06:17:59|   6| 15|  46|   11|20

In [94]:
time.write.mode("overwrite").parquet("time.parquet")
time_par = spark.read.parquet('time.parquet')
time_par.select('*').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----+---+----+-----+----+-------+
|start_time|hour|day|week|month|year|weekday|
+----------+----+---+----+-----+----+-------+
|  00:30:26|   0| 15|  46|   11|2018|      5|
|  00:41:21|   0| 15|  46|   11|2018|      5|
|  00:45:41|   0| 15|  46|   11|2018|      5|
|  01:57:51|   1| 15|  46|   11|2018|      5|
|  03:29:37|   3| 15|  46|   11|2018|      5|
|  03:44:09|   3| 15|  46|   11|2018|      5|
|  03:44:20|   3| 15|  46|   11|2018|      5|
|  05:34:34|   5| 15|  46|   11|2018|      5|
|  05:37:57|   5| 15|  46|   11|2018|      5|
|  05:48:55|   5| 15|  46|   11|2018|      5|
|  05:53:44|   5| 15|  46|   11|2018|      5|
|  05:55:56|   5| 15|  46|   11|2018|      5|
|  06:01:02|   6| 15|  46|   11|2018|      5|
|  06:01:53|   6| 15|  46|   11|2018|      5|
|  06:07:37|   6| 15|  46|   11|2018|      5|
|  06:10:33|   6| 15|  46|   11|2018|      5|
|  06:13:54|   6| 15|  46|   11|2018|      5|
|  06:14:16|   6| 15|  46|   11|2018|      5|
|  06:17:59|   6| 15|  46|   11|20